In [1]:
import pandas as pd
import os
import numpy as np
import subprocess
from tqdm import tqdm
from itertools import combinations
from collections import defaultdict
import itertools
import random

In [2]:
def generate_mixed_fastqs(list_of_fastqs, fpath_fqi, fpath_fqo, fpath_index, nsamples=1000, mode='train'):

    def combine_fastqs(sample_props, fqd):

        def read_fastq_seqs(filepath, prop, ct):

            lines_list = []
            with open(filepath, 'r') as fh:
                for seq_header, seq, qual_header, qual in itertools.zip_longest(*[fh] * 4):
                    
                    if any(line is None for line in (seq_header, seq, qual_header, qual)):
                        raise Exception(
                            "Number of lines in FASTQ file must be multiple of four "
                            "(i.e., each record must be exactly four lines long).")
                    if not seq_header.startswith('@'):
                        raise Exception("Invalid FASTQ sequence header: %r" % seq_header)
                    if qual_header != '+\n':
                        raise Exception("Invalid FASTQ quality header: %r" % qual_header)
                    if qual == '\n':
                        raise Exception("FASTQ record is missing quality scores.")

                    lines_list.append("".join([seq_header, seq, qual_header, qual]))

            #randomly select lines according to proportions
            if prop < 1:
                n = int(prop * len(lines_list))
                subsampled_lines_list = random.sample(lines_list, n)
                print(f"{n} lines sampled from fastq, from {prop} proportions of {ct} celltype")
            else:
                print(f"all lines sampled from fastq, from proportions {ct}")

            return subsampled_lines_list

        def select_fastq_piece(name, prop, ct):
            fastq_lines = read_fastq_seqs(os.path.join(fpath_fqi,f'{name}.fastq'), prop, ct)
            return fastq_lines

        mixedfastq = []
        names = []
        for ct, prop in tqdm(sample_props, 'mixing fastq according to proportions'): #iterate celltype-proportion tuplas

            if prop > 0:
                name = random.sample(fqd[ct],1)[0]
                names.append(name)
                mixedfastq += select_fastq_piece(name, prop, ct)

        #write fastq
        #generate a random id
        rid = ''.join(map(str,random.sample(range(1000),4)))

        #write the rid-names map
        with open(os.path.join(fpath_fqo, mode, f'mixed_{mode}_names_list.csv'), 'a') as file:
            file.write(','.join([rid, '||'.join(names)]) + '\n')

        #retrieve proportions
        proportions_str = list(map(lambda x: str(x[1]), sample_props))

        with open(os.path.join(fpath_fqo, mode, f'mixed_{mode}_proportions.csv'), 'a') as file:
            file.write(','.join(proportions_str) + '\n')

        ## now save the fastq
        with open(os.path.join(fpath_fqo, mode, 'samples',f'mixed_{mode}_sample_id{rid}.fastq'), 'w') as file:
            for line in mixedfastq:
                file.write(line)

        #now quantify (generate the count matrix for the fastq)
        kallisto_quant = f'kallisto quant -i {fpath_index} -o {os.path.join(fpath_fqo, mode, "samples")}/mixed_{mode}_sample_id{rid}_counts --single -l 200 -s 20 -t 64 {os.path.join(fpath_fqo, mode, "samples", f"mixed_{mode}_sample_id{rid}.fastq")}'
        subprocess.call(kallisto_quant, shell=True)

        ## now remove the fastq file
        os.remove(os.path.join(fpath_fqo,mode,'samples',f'mixed_{mode}_sample_id{rid}.fastq'))

    #define celltypes
    celltypes = ["ct1", "ct2"]

    ## generate dict of fastqs according to the celltype
    fqd = defaultdict(list)
    for f in list_of_fastqs:
        for ct in celltypes:
            if ct in f:
                fqd[ct].append(f)

    nct = len(celltypes)

    ## init empty list to fill it with generated combinations
    combl = []
    s = 1
    for i in range(1, nct+1): ## group of combinations
        if i > 1:
            s = 10
        cmb = list(combinations(range(nct),i))
        for e in cmb:
            combelm = np.zeros((s, nct))
            combelm[:,e] = np.random.dirichlet(alpha = np.ones(len(e)), size = s)
            combl.append(combelm)

    ## generate benchmarking proportions
    benchmark_props = pd.DataFrame(np.vstack(combl))
    benchmark_props.columns = celltypes

    ## go through the matrix, until n samples have been generated
    n = 0
    while n < nsamples:

        if not (n%5):
            print(f"number of samples {n}")

        for i in range(benchmark_props.shape[0]):

            sample_props = list(zip(celltypes, benchmark_props.iloc[i,:]))
            combine_fastqs(sample_props, fqd)
            n+=1



In [3]:
## define path to a folder with fastqs files to perform the pseudobulk generation
## this example is for single-ended fastqs, modify accordingly in the case of paired-ended 
fpath_fqi = os.path.join('../','data','fastqs_input')
list_of_fastqs = os.listdir(fpath_fqi)

fpath_fqo = os.path.join('../','data','fastqs_output')

if not os.path.isdir(os.path.join(fpath_fqo,'train')):
    os.mkdir(os.path.join(fpath_fqo,'train'))

if not os.path.isdir(os.path.join(fpath_fqo, 'test')):
    os.mkdir(os.path.join(fpath_fqo, 'test'))

## we would need also the index of the transcriptome (Homo_sapiens.GRCh38)
fpath_index = os.path.join('../', 'data','transcriptome.idx')

#now, separate train and test
train_files = [f for f in list_of_fastqs if 'file3' not in f]
test_files = [f for f in list_of_fastqs if 'file3' in f]

print(f"fastq files {list_of_fastqs},\ntrain {train_files},\ntest {test_files}")

## generate mixed fastqs for train and test files
#generate_mixed_fastqs(train_files, fpath_fqi, fpath_fqo, fpath_index, nsamples = 100, mode ='train')
#generate_mixed_fastqs(test_files, fpath_fqi, fpath_fqo, fpath_index, nsamples = 10, mode ='test')

fastq files ['file1_ct2.fastq.gz', 'file1_ct1.fastq.gz', 'file2_ct1.fastq.gz', 'file3_ct1.fastq.gz', 'file3_ct2.fastq.gz', 'file2_ct2.fastq.gz'],
train ['file1_ct2.fastq.gz', 'file1_ct1.fastq.gz', 'file2_ct1.fastq.gz', 'file2_ct2.fastq.gz'],
test ['file3_ct1.fastq.gz', 'file3_ct2.fastq.gz']
